In [71]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
import folium
from folium.plugins import FastMarkerCluster

from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

In [116]:
datum = "EPSG:4326"
long = -79.2241
lat = 45.4305
earth_circumference = 40074 #km
lat_radians = lat * (np.pi/180) #conversion of degrees to radians
long_km = 360 / (earth_circumference * np.cos(lat_radians)) 
lat_km = 1/110.574 
n=25
shift = 50 
#portion = 1

#Constant function of '1s' at every spatial location. Homogeneous.
def ones_2d(size=n):
    return np.ones((n,n))

# Random normally distributed values, can set the mean and standard deviation. Can be scaled with the parameter A.
def normal_dist_2d(mean=0,sd=1,size=n,A=1):
    return A*np.random.normal(mean,sd, size =(n,n))

# https://en.wikipedia.org/wiki/Gaussian_function#Two-dimensional_Gaussian_function  
def gaus2d(x=0, y=0, mx=long, my=lat, sx=0.009, sy=0.009,shift_x=0.01,shift_y=0.00):
    # Different amplitude parameters, wikipedia uses A=1, however the chosen amplitude is usually selected
      # for gaussian normalization (integral = 1)
    A = 1. / (2 * np.pi*sx * sy)
    
    return A * np.exp(-((x - mx - shift_x)**2. / (2. * sx**2.) + (y - my - shift_y)**2. / (2. * sy**2.)))


In [117]:

def to_esri_shapefile(z,portion=1):
    df = pd.DataFrame(z,index = x, columns=y)
    newdf = df.unstack().reset_index().rename(columns={'level_0':'longitude','level_1':'lat',0:'target_variable'})
    # Which portion of the data do we want to retain (1 is the entire set)
    newdf = newdf.sample(frac=portion)
    #Create a geopandas geodataframe to begin transition to .shp file. Assign Point Data.
    gdf = gpd.GeoDataFrame(newdf, geometry=gpd.points_from_xy(newdf.longitude, newdf.lat))
    #Define a Geodetic Parameter. EPSG 4326 refers to WGS84 geodetic datum.
    gdf.crs = datum
    #To export as a shapefile:
    #%insights_return(gdf)
    #gdf.to_file(driver = 'ESRI Shapefile', filename= "./test_data/test_1/normal_dist.shp") 
    
    return gdf

In [118]:
x = np.linspace(lat - shift*lat_km, lat + shift*lat_km,n, endpoint=True)
y = np.linspace(long - shift*long_km,long + shift*long_km,n, endpoint=True)
xx , yy = np.meshgrid(x,y,indexing='ij')

###
### Functions
###

z = ones_2d(n)

In [119]:
z

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 

In [135]:
gdf = to_esri_shapefile(z,portion=1)
gdf_sample_50 = to_esri_shapefile(z,portion=0.2)

In [136]:
gdf

,longitude,lat,target_variable,geometry
518,-78.797401,45.656593,1.0,POINT (-78.79740 45.65659)
603,-78.584052,45.091361,1.0,POINT (-78.58405 45.09136)
257,-79.330775,45.242089,1.0,POINT (-79.33077 45.24209)
281,-79.277437,45.204407,1.0,POINT (-79.27744 45.20441)
617,-78.584052,45.618911,1.0,POINT (-78.58405 45.61891)
...,...,...,...,...
551,-78.690727,45.015996,1.0,POINT (-78.69073 45.01600)
452,-78.904076,45.053678,1.0,POINT (-78.90408 45.05368)
169,-79.544124,45.694275,1.0,POINT (-79.54412 45.69428)
446,-78.957413,45.769639,1.0,POINT (-78.95741 45.76964)


In [137]:
gdf_sample_50

,longitude,lat,target_variable,geometry
551,-78.690727,45.015996,1.0,POINT (-78.69073 45.01600)
336,-79.170763,45.392818,1.0,POINT (-79.17076 45.39282)
246,-79.384112,45.769639,1.0,POINT (-79.38411 45.76964)
80,-79.704136,45.166725,1.0,POINT (-79.70414 45.16672)
111,-79.650799,45.392818,1.0,POINT (-79.65080 45.39282)
...,...,...,...,...
157,-79.544124,45.242089,1.0,POINT (-79.54412 45.24209)
141,-79.597461,45.581229,1.0,POINT (-79.59746 45.58123)
471,-78.904076,45.769639,1.0,POINT (-78.90408 45.76964)
569,-78.690727,45.694275,1.0,POINT (-78.69073 45.69428)


In [138]:
my_map_complete = folium.Map(location=[lat,long], zoom_start=10)

folium.GeoJson(data = gdf,popup=folium.GeoJsonPopup(fields=["target_variable",
                                                                     "lat",
                                                                     "longitude",
                                                                     ])).add_to(my_map_complete)

In [139]:
my_map_complete

In [140]:

my_map_sample_50 = folium.Map(location=[lat,long], zoom_start=10)

folium.GeoJson(data = gdf_sample_50,popup=folium.GeoJsonPopup(fields=["target_variable",
                                                                     "lat",
                                                                     "longitude",
                                                                     ])).add_to(my_map_sample_50)

In [141]:
my_map_sample_50

In [142]:
gdf_numpy = gdf[["longitude","lat"]].to_numpy()

gdf_numpy_sample = gdf_sample_50[["longitude","lat"]].to_numpy()



In [143]:
gdf_numpy

array([[-78.79740129,  45.65659293],
       [-78.58405194,  45.0913606 ],
       [-79.33077468,  45.24208922],
       ...,
       [-79.54412403,  45.69427509],
       [-78.95741331,  45.7696394 ],
       [-79.59746137,  45.35513569]])

In [157]:
kms_per_radian = 6371.0088
epsilon = 2.5 / kms_per_radian
db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(gdf_numpy_sample))
cluster_labels = db.labels_
num_clusters = len(set(cluster_labels))
clusters = pd.Series([gdf_numpy_sample[cluster_labels == n] for n in range(num_clusters)])

Number of points: 125
Number of clusters: 71


This function returns the center-most point from a cluster by taking a set of points (i.e., a cluster) and returning the point within it that is nearest to some reference point (in this case, the cluster’s centroid).
The function above first calculates the centroid’s coordinates. Then I use Python’s built-in min function to find the smallest member of the cluster in terms of distance to that centroid. The key argument does this with a lambda function that calculates each point’s distance to the centroid in meters, via geopy’s great circle function. Finally, I return the coordinates of the point that was the least distance from the centroid.

In [158]:
def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)
centermost_points = clusters.map(get_centermost_point)

In [159]:
centermost_points[0]

(-78.69072661822241, 45.01599628906132)

In [160]:
gdf_numpy_sample

array([[-78.69072662,  45.01599629],
       [-79.17076266,  45.39281784],
       [-79.38411201,  45.7696394 ],
       [-79.70413604,  45.16672491],
       [-79.65079871,  45.39281784],
       [-79.2241    ,  45.35513569],
       [-79.43744935,  45.65659293],
       [-79.49078669,  45.58122862],
       [-79.01075065,  45.31745353],
       [-79.38411201,  45.58122862],
       [-78.74406396,  45.7696394 ],
       [-79.38411201,  45.46818216],
       [-79.17076266,  45.69427509],
       [-79.65079871,  45.69427509],
       [-78.79740129,  45.50586431],
       [-78.95741331,  45.80732156],
       [-79.43744935,  45.61891078],
       [-79.65079871,  44.97831413],
       [-78.63738928,  45.16672491],
       [-79.33077468,  45.84500371],
       [-79.2241    ,  45.4305    ],
       [-79.27743734,  45.16672491],
       [-79.2241    ,  45.73195724],
       [-79.17076266,  44.97831413],
       [-79.2241    ,  45.61891078],
       [-79.65079871,  45.27977138],
       [-78.90407597,  45.24208922],
 

## Cluster centers are blue circles and the data set is in red markers

In [164]:
print("The data reduction of {} points to {} cluster centers.".format(len(gdf_sample_50),num_clusters))

The data reduction of 125 points to 71 cluster centers.


In [166]:
my_map_clusters = folium.Map(location=[lat,long], zoom_start=10)


for point in centermost_points :
    loc = [point[1],point[0]]
    #folium.Marker(location=loc,icon=folium.Icon(color="red")).add_to(my_map_clusters)
    folium.Circle(radius=2000,location=loc,color="BLUE").add_to(my_map_clusters)

for point in gdf_numpy_sample :
    loc = [point[1],point[0]]
    folium.Marker(location=loc,icon=folium.Icon(color="red")).add_to(my_map_clusters)



my_map_clusters